# VacationPy
___

In [1]:
# dependencies and setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load csv created in WeatherPy into a pandas dataframe
city_data_df = pd.read_csv("output_data/cities.csv")

# display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,9.07,78,100,3.70,AU,1722617405
1,1,sirte,31.2089,16.5887,28.50,61,0,4.58,LY,1722617406
2,2,nadym,65.5333,72.5167,20.03,56,43,4.79,RU,1722617407
3,3,canete,-37.8000,-73.4000,11.36,93,80,3.43,CL,1722617177
4,4,invercargill,-46.4000,168.3500,-1.66,97,17,0.89,NZ,1722617216


___
#### Step 1: Create map that displays point for every city in `city_data_df` DataFrame. The size of every city is the humidity in that city on 2024-08-02

In [3]:
%%capture --no-display

# help from https://hvplot.holoviz.org/reference/geopandas/points.html
# and https://hvplot.holoviz.org/user_guide/Customization.html
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles=True,
    color="City",
    size="Humidity",
    alpha=0.5,
    legend=False,
    frame_width = 700,
    frame_height = 500)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

#### Step 2: Narrow down `city_data_df` to find ideal weather conditions

In [4]:
# drop null values from city_data_df
city_data_df.dropna(inplace=True)

# create new ideal_cities_df, filter out cloudy places
ideal_cities_df = city_data_df.loc[city_data_df["Cloudiness"] <= 10]

# filter out places with temperatures below 15 degrees Celsius
ideal_cities_df = ideal_cities_df.loc[ideal_cities_df["Max Temp"] >= 15]

# display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,sirte,31.2089,16.5887,28.50,61,0,4.58,LY,1722617406
37,37,pierre,44.3683,-100.3510,34.60,43,0,2.06,US,1722617432
38,38,casper,42.8666,-106.3131,31.01,29,0,2.57,US,1722617293
50,50,bafq,31.6128,55.4107,38.92,10,8,6.50,IR,1722617467
52,52,constantia,44.1833,28.6500,30.38,58,6,8.52,RO,1722617060


#### Step 3: Create a new `hotel_df` DataFrame

In [5]:
# create hotel_df DataFrame that only contains cities, lat, lng, and humidity from ideal_cities_df
hotel_df = ideal_cities_df[["City", "Lat", "Lng", "Humidity"]].copy()

# add empy "Hotel Name" column
hotel_df["Hotel Name"] = ""

# display sample
hotel_df.head()

,City,Lat,Lng,Humidity,Hotel Name
1,sirte,31.2089,16.5887,61,
37,pierre,44.3683,-100.3510,43,
38,casper,42.8666,-106.3131,29,
50,bafq,31.6128,55.4107,10,
52,constantia,44.1833,28.6500,58,


#### Step 4: For each city, use Geoapify API to find the first hotel located within 10,000 meters of city

In [6]:
# set base url
base_url = "https://api.geoapify.com/v2/places?"

# set parameters to search for a hotel
radius = 10000
params = {
    "limit": "1",
    "apiKey": {geoapify_key},
    "categories": "accommodation.hotel"
}

# print a message to acknowledge that the hotel search is starting
print("Starting hotel search")

# iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get lat and lon ffrom row
    lat = row["Lat"]
    lon = row["Lng"]

    # add current city's latitude and longitude to params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # make api request and convert it to json
    response = requests.get(base_url, params=params).json()

    # grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    
    except (KeyError, IndexError):
        # if no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# display the sample data
hotel_df.head()

Starting hotel search
sirte - nearest hotel: فندق المدينة-سرت
pierre - nearest hotel: Quality Inn
casper - nearest hotel: Days Inn by Wyndham Casper
bafq - nearest hotel: هتل رستوران علمدار
constantia - nearest hotel: Hotel Dali
saint croix - nearest hotel: The Buccaneer
iseo - nearest hotel: Ambra Hotel
fale old settlement - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
bijar - nearest hotel: خانه معلم
albufeira - nearest hotel: Hotel Baltum
lata - nearest hotel: Residencial Botânico
naze - nearest hotel: ビジネス旅館畠山
robbio - nearest hotel: Hotel Novarello
nanning - nearest hotel: No hotel found
eastham - nearest hotel: Four Points by Sheraton
nassau - nearest hotel: Hilton Garden Inn
muynoq - nearest hotel: Lala
placido de castro - nearest hotel: No hotel found
inhambane - nearest hotel: Hotel de Inhambane
port mathurin - nearest hotel: Escale Vacances
antalaha - nearest hotel: Hôtel Florida
ramadi - nearest hotel: فندق الانبار الدولي
kas -

,City,Lat,Lng,Humidity,Hotel Name
1,sirte,31.2089,16.5887,61,فندق المدينة-سرت
37,pierre,44.3683,-100.3510,43,Quality Inn
38,casper,42.8666,-106.3131,29,Days Inn by Wyndham Casper
50,bafq,31.6128,55.4107,10,هتل رستوران علمدار
52,constantia,44.1833,28.6500,58,Hotel Dali


#### Step 5: Create a new map only with ideal cities that contains the hotel name as additional information

In [7]:
%%capture --no-display

map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles=True,
    color="City",
    size="Humidity",
    alpha=0.5,
    legend=False,
    hover_cols=["Hotel Name"],
    frame_width = 700,
    frame_height = 500)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)